## Técnicas de optimización en Spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark Optimizacion") \
    .getOrCreate()


23/09/25 20:10:24 WARN Utils: Your hostname, MacBook-Air-de-Ivan.local resolves to a loopback address: 127.0.0.1; using 192.168.0.2 instead (on interface en0)
23/09/25 20:10:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 20:10:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/25 20:10:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Apache Spark es conocido por ser un sistema de computación en clúster rápido y general. Aunque es eficiente por diseño, hay varias técnicas que los desarrolladores pueden emplear para maximizar el rendimiento y aprovechar al máximo Spark.

La optimización en Spark se centra en mejorar la eficiencia y la velocidad de procesamiento de las operaciones, minimizando los recursos utilizados y el tiempo de espera. Esto es crucial cuando se manejan grandes conjuntos de datos y se realizan operaciones intensivas. A continuación, discutiremos algunas de las técnicas comunes de optimización en Spark.


### Uso de Broadcast Variables

Las variables de transmisión permiten mantener una variable de solo lectura en caché en cada máquina en lugar de enviar una copia de ella con las tareas. Son útiles cuando las tareas a través de múltiples etapas necesitan la misma variable de solo lectura.

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

spark = SparkSession.builder.appName("Optimization Techniques").getOrCreate()

# Creando DataFrames de ejemplo
data_large = [("A", 10), ("B", 20), ("C", 30), ("D", 40)]
data_small = [("A", 1), ("B", 2)]
df_large = spark.createDataFrame(data_large, ["id", "value1"])
df_small = spark.createDataFrame(data_small, ["id", "value2"])

# Usando broadcast para el DataFrame más pequeño durante la operación join
joined_df = df_large.join(broadcast(df_small), "id")
joined_df.show()


23/09/25 21:49:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---+------+------+
| id|value1|value2|
+---+------+------+
|  A|    10|     1|
|  B|    20|     2|
+---+------+------+



En el ejemplo anterior, primero creamos dos DataFrames: uno grande (`df_large`) y uno pequeño (`df_small`). Al hacer un `join` entre ellos, utilizamos la función `broadcast` en el DataFrame más pequeño. Esto asegura que el DataFrame pequeño se envíe a todos los nodos solo una vez, permitiendo que Spark realice el join de manera más eficiente.


### Uso de Data Partitioning

El particionado es una técnica para distribuir equitativamente los datos a través de las particiones para optimizar la paralelización y minimizar la cantidad de datos que se envían a través de la red.

In [7]:
# Creando un RDD de ejemplo
rdd = spark.sparkContext.parallelize([(1, "A", 100), (2, "B", 200), (3, "C", 300), (4, "D", 400)], 2)

# Transformando el RDD a DataFrame
df = spark.createDataFrame(rdd, ["id", "name", "value"])

# Mostrar el número de particiones antes del reparticionado
print("Número de particiones antes del reparticionado:", df.rdd.getNumPartitions())

# Reparticionando el DataFrame en 4 particiones
df_repartitioned = df.repartition(4)

# Mostrar el número de particiones después del reparticionado
print("Número de particiones después del reparticionado:", df_repartitioned.rdd.getNumPartitions())


Número de particiones antes del reparticionado: 2
Número de particiones después del reparticionado: 4


En este ejemplo, primero creamos un RDD llamado `rdd` con 2 particiones. Después, convertimos ese RDD en un DataFrame (`df`). A continuación, mostramos el número de particiones antes y después de reparticionar el DataFrame. Usamos el método `repartition(4)` para cambiar el número de particiones a 4. El reparticionado puede ser útil para distribuir los datos de manera uniforme y optimizar las operaciones paralelas.


### Configuración y ajuste del rendimiento

Spark ofrece múltiples configuraciones que pueden ser ajustadas para optimizar el rendimiento según las características del trabajo y la arquitectura del clúster.

### Configuración de Memoria

La memoria en Spark se divide en dos regiones: memoria de ejecución y memoria de almacenamiento. La configuración adecuada de la memoria puede mejorar significativamente el rendimiento de las aplicaciones Spark.

In [9]:
def get_spark_conf(key, default="Not Defined"):
    try:
        return spark.conf.get(key)
    except:
        return default

# Mostrar la configuración actual de memoria
spark_memory_conf = {
    "Memory per node": get_spark_conf("spark.driver.memory"),
    "Max result size": get_spark_conf("spark.driver.maxResultSize"),
    "Memory fraction": get_spark_conf("spark.memory.fraction"),
    "Storage memory fraction": get_spark_conf("spark.memory.storageFraction")
}

for key, value in spark_memory_conf.items():
    print(key + ": " + value)



Memory per node: Not Defined
Max result size: Not Defined
Memory fraction: Not Defined
Storage memory fraction: Not Defined



La configuración de memoria en Spark es esencial para garantizar que tus trabajos se ejecuten de manera eficiente y sin errores relacionados con la memoria. Aunque no todas las configuraciones están explícitamente definidas en todos los entornos de Spark, es importante conocerlas y, si es necesario, ajustarlas según las necesidades de tu trabajo y los recursos disponibles.

Aquí están las configuraciones de memoria que intentamos obtener, junto con sus valores predeterminados en Spark (estos valores pueden variar según la versión de Spark y la configuración del clúster):

- **Memory per node (spark.driver.memory)**: Memoria asignada por nodo. Valor predeterminado: 1g.
- **Max result size (spark.driver.maxResultSize)**: Tamaño máximo de resultados que se pueden recopilar en el controlador. Valor predeterminado: 1g.
- **Memory fraction (spark.memory.fraction)**: Fracción del heap de Java destinado a Spark. Valor predeterminado: 0.6.
- **Storage memory fraction (spark.memory.storageFraction)**: Fracción del heap de memoria de Spark destinado al almacenamiento. Valor predeterminado: 0.5.

Para obtener más detalles sobre estas configuraciones y otros ajustes relacionados con la memoria, consulta la [documentación oficial de Spark](https://spark.apache.org/docs/latest/configuration.html#application-properties).



### Optimización de la Serialización

La serialización es el proceso de convertir un objeto en un flujo de bytes para su almacenamiento o transmisión. 

Spark utiliza la serialización para:

Transmitir datos a través de la red durante operaciones de shuffle.

Almacenar datos en memoria.

Spark proporciona dos bibliotecas de serialización:

Java (predeterminada): Proporciona un buen equilibrio entre velocidad y serialización de cualquier objeto. Sin embargo, los datos serializados suelen ser más grandes, lo que puede aumentar el tiempo de transmisión y el uso de la memoria.

Kryo: Es más rápido y produce datos serializados más pequeños, pero no admite todos los tipos de objetos por defecto.

Para configurar Spark para usar Kryo:

In [ ]:
spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")


### Evitar Operaciones Costosas

Ejemplo:

Supongamos que tenemos el siguiente RDD y queremos sumar valores por clave:


In [11]:
rdd = spark.sparkContext.parallelize([(1, 2), (3, 4), (3, 6)])

En lugar de usar `groupByKey`

In [12]:
sums = rdd.groupByKey().mapValues(lambda x: sum(x))

Es más eficiente usar `reduceByKey`


In [13]:
sums = rdd.reduceByKey(lambda x, y: x + y)


### Caché y Persistencia

Si vamos a utilizar un DataFrame o RDD múltiples veces, podemos beneficiarnos del uso de `cache()` o `persist()`.

Ejemplo:

Supongamos que tenemos un DataFrame y queremos realizar varias operaciones:

In [16]:
from pyspark.sql import Row

data = [Row(id=i, value=i**2) for i in range(100)]
df = spark.createDataFrame(data)



Podemos cachearlo para acelerar operaciones posteriores

In [18]:
df.cache()

# Operaciones posteriores se benefician del caché
df.filter("id < 50").show()


df.groupBy("id").count().show()


23/09/25 22:11:23 WARN CacheManager: Asked to cache already cached data.


+---+-----+
| id|value|
+---+-----+
|  0|    0|
|  1|    1|
|  2|    4|
|  3|    9|
|  4|   16|
|  5|   25|
|  6|   36|
|  7|   49|
|  8|   64|
|  9|   81|
| 10|  100|
| 11|  121|
| 12|  144|
| 13|  169|
| 14|  196|
| 15|  225|
| 16|  256|
| 17|  289|
| 18|  324|
| 19|  361|
+---+-----+
only showing top 20 rows



+---+-----+
| id|count|
+---+-----+
|  0|    1|
|  7|    1|
|  6|    1|
|  9|    1|
|  5|    1|
|  1|    1|
| 10|    1|
|  3|    1|
|  8|    1|
| 11|    1|
|  2|    1|
|  4|    1|
| 19|    1|
| 22|    1|
| 17|    1|
| 12|    1|
| 13|    1|
| 18|    1|
| 14|    1|
| 21|    1|
+---+-----+
only showing top 20 rows



Cuando aplicamos transformaciones y acciones a df, como el filtro, la muestra y el groupBy, si no hubiéramos utilizado df.cache(), Spark podría haber recalculado df desde el principio cada vez. Esto podría haber sido especialmente costoso si df se hubiera derivado de una serie de transformaciones complejas.

Sin embargo, dado que usamos df.cache(), después de que df se calculó y se cargó en la memoria la primera vez, todas las acciones subsiguientes (filtrado, muestra, etc.) se realizaron mucho más rápido, ya que utilizaron la versión en caché de df.

Cada vez que visualizamos o interactuamos con el DataFrame df, estamos aprovechando el beneficio del caché. Si df fuera grande y derivado de múltiples operaciones, el beneficio de usar caché sería aún más notorio en términos de rendimiento.

### Monitorizar y Utilizar la UI de Spark

La interfaz de usuario de Spark (Spark UI) es una herramienta esencial para la monitorización y depuración de aplicaciones Spark. Proporciona una visión detallada de la ejecución de trabajos, etapas y tareas, así como métricas de rendimiento y detalles sobre el uso de recursos.

#### Acceder a la Spark UI

Por defecto, Spark UI está habilitada y se puede acceder a ella a través de un navegador web en el puerto 4040 de la máquina donde se ejecuta el controlador de Spark. La URL suele ser http://[driver-node]:4040.

#### Componentes principales de Spark UI:
1. Página de resumen (Jobs)
Aquí se muestra una lista de trabajos ejecutados, en ejecución y pendientes. Puedes ver detalles como el ID del trabajo, la duración, las etapas y las tareas asociadas, así como el estado del trabajo (completado, en ejecución, fallido).

2. Stages
Esta pestaña muestra detalles sobre las etapas de los trabajos. Una etapa es una unidad de trabajo en Spark, y cada trabajo puede tener múltiples etapas. Puedes ver detalles como el número de tareas, la duración, los datos leídos, y los bytes y registros procesados.

3. Storage
Muestra información sobre los RDDs y DataFrames almacenados en memoria o en disco. Es útil para monitorizar el uso de la memoria y el almacenamiento en caché.

4. Environment
Proporciona información sobre la configuración de Spark y los detalles del entorno. Es útil para verificar y depurar la configuración de Spark.

5. Executors
Muestra detalles sobre los ejecutores, que son procesos en nodos de trabajador que ejecutan tareas. Puedes ver métricas como la memoria utilizada, la CPU y el número de tareas completadas.

6. SQL
Si estás ejecutando consultas Spark SQL, esta pestaña muestra detalles sobre la ejecución de esas consultas, incluidos los planes de ejecución físicos y lógicos.

#### Ejemplo práctico:


In [ ]:
# Supongamos que tenemos el siguiente código que lee un archivo CSV y realiza algunas transformaciones.
data = spark.read.csv("path_to_file.csv", header=True, inferSchema=True)
data.groupBy("some_column").count().show()

# Después de ejecutar este código, puedes ir a Spark UI y:
# 1. Navega a la pestaña "Jobs" para ver el trabajo ejecutado.
# 2. Haz clic en el ID del trabajo para ver detalles sobre las etapas y tareas.
# 3. Navega a la pestaña "Stages" para ver más detalles sobre las etapas.
# 4. Si almacenaste algo en caché, verifica la pestaña "Storage" para ver su tamaño y ubicación (memoria o disco).


#### Recomendaciones para usar Spark UI:

Depuración de errores: Si un trabajo o etapa falla, Spark UI es el primer lugar donde debes buscar detalles sobre el error.

Optimización: Si notas que un trabajo está llevando más tiempo del esperado, verifica las etapas y tareas para identificar cuellos de botella.

Monitorización del uso de recursos: Verifica el uso de memoria y CPU en la pestaña "Executors" para asegurarte de que tu aplicación está utilizando los recursos de manera eficiente.

La UI de Spark es una herramienta esencial para cualquier desarrollador o administrador de Spark. Proporciona una gran cantidad de información que puede ayudarte a optimizar y depurar tus aplicaciones Spark. Es una buena práctica familiarizarse con ella y consultarla regularmente mientras desarrollas y ejecutas aplicaciones Spark.